In [38]:
# The code was removed by DSX for sharing.

In [39]:
from io import BytesIO  
import requests  
import json  
import pandas as pd
from io import BytesIO  
import requests  
import json  
import pandas as pd

def put_file(credentials, local_file_name):  
    """This functions returns a StringIO object containing
    the file content from Bluemix Object Storage V3."""
    f = open(local_file_name,'r')
    my_data = f.read()
    url1 = ''.join(['https://identity.open.softlayer.com', '/v3/auth/tokens'])
    data = {'auth': {'identity': {'methods': ['password'],
            'password': {'user': {'name': credentials['username'],'domain': {'id': credentials['domain_id']},
            'password': credentials['password']}}}}}
    headers1 = {'Content-Type': 'application/json'}
    resp1 = requests.post(url=url1, data=json.dumps(data), headers=headers1)
    resp1_body = resp1.json()
    for e1 in resp1_body['token']['catalog']:
        if(e1['type']=='object-store'):
            for e2 in e1['endpoints']:
                        if(e2['interface']=='public'and e2['region']=='dallas'):
                            url2 = ''.join([e2['url'],'/', credentials['container'], '/', local_file_name])
    s_subject_token = resp1.headers['x-subject-token']
    headers2 = {'X-Auth-Token': s_subject_token, 'accept': 'application/json'}
    resp2 = requests.put(url=url2, headers=headers2, data = my_data )
    print resp2

In [40]:
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import LabelEncoder
import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, make_scorer

In [41]:
train = pd.read_csv(get_object_storage_file_with_credentials_9b3c597dc9074fdc8a1dbb2f44d73d63('DefaultProjectlakshmi1310nagagmailcom', 'train.csv'))
test = pd.read_csv(get_object_storage_file_with_credentials_9b3c597dc9074fdc8a1dbb2f44d73d63('DefaultProjectlakshmi1310nagagmailcom', 'test.csv'))

In [42]:
print train.isnull().sum()
print train.Is_Response.value_counts().apply(lambda x:x*1./len(train))

User_ID         0
Description     0
Browser_Used    0
Device_Used     0
Is_Response     0
dtype: int64
happy        0.681213
not happy    0.318787
Name: Is_Response, dtype: float64


Above values show that data set is imbalance problem and has no null values

Creating xgboost model with only 2 features 

In [44]:
# The code was removed by DSX for sharing.

In [45]:
# function to clean data

stops = set(stopwords.words("english"))
def cleanData(text, lowercase = False, remove_stops = False, stemming = False):
    txt = str(text)
    txt = re.sub(r'[^A-Za-z0-9\s]',r'',txt)
    txt = re.sub(r'\n',r' ',txt)
    
    if lowercase:
        txt = " ".join([w.lower() for w in txt.split()])
        
    if remove_stops:
        txt = " ".join([w for w in txt.split() if w not in stops])
    
    if stemming:
        st = PorterStemmer()
        txt = " ".join([st.stem(w) for w in txt.split()])

    return txt

In [46]:
test['Is_Response'] = np.nan
df = train.append(test)
print(df.shape)

(68336, 5)


In [47]:
# clean description
df['Description'] = df['Description'].map(lambda x: cleanData(x, lowercase=True, remove_stops=True, stemming=True))

In [48]:


# initialise the functions - we'll create separate models for each type.
countvec = CountVectorizer(analyzer='word', ngram_range = (1,1), min_df=150, max_features=500)
tfidfvec = TfidfVectorizer(analyzer='word', ngram_range = (1,1), min_df = 150, max_features=500)



In [49]:
bagofwords = countvec.fit_transform(df['Description'])
tfidfdata = tfidfvec.fit_transform(df['Description'])

In [50]:
# label encode categorical features in data given
cols = ['Browser_Used','Device_Used']

for x in cols:
    lbl = LabelEncoder()
    df[x] = lbl.fit_transform(df[x])

In [51]:
# create dataframe for features
bow_df = pd.DataFrame(bagofwords.todense())
tfidf_df = pd.DataFrame(tfidfdata.todense())


In [52]:
# set column names
bow_df.columns = ['col'+ str(x) for x in bow_df.columns]
tfidf_df.columns = ['col' + str(x) for x in tfidf_df.columns]


In [53]:
# create separate data frame for bag of words and tf-idf

bow_df_train = bow_df[:len(train)]
bow_df_test = bow_df[len(train):]

tfid_df_train = tfidf_df[:len(train)]
tfid_df_test = tfidf_df[len(train):]


In [54]:
# split the merged data file into train and test respectively
train_feats = df[~pd.isnull(df.Is_Response)]
test_feats = df[pd.isnull(df.Is_Response)]

In [55]:
train_feats['Is_Response'] = [1 if x == 'happy' else 0 for x in train_feats['Is_Response']]

/usr/local/src/bluemix_jupyter_bundle.v59/notebook/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [56]:
# merge count (bag of word) features into train
bow_train = pd.concat([train_feats[cols], bow_df_train], axis = 1)
bow_test = pd.concat([test_feats[cols], bow_df_test], axis=1)

bow_test.reset_index(drop=True, inplace=True)

In [57]:
# merge into a new data frame with tf-idf features
train_feats2 = pd.concat([train_feats[cols], tfid_df_train], axis=1)
test_feats2 = pd.concat([test_feats[cols], tfid_df_test], axis=1)

In [58]:
bow_train.to_csv('bow_train.csv',index=False)
bow_test.to_csv('bow_test.csv',index=False)
train_feats2.to_csv('tfid_train.csv',index=False)
test_feats2.to_csv('tfid_train.csv',index=False)

In [59]:
put_file(credentials_10,'bow_train.csv')
put_file(credentials_10,'bow_test.csv')
put_file(credentials_10,'tfid_train.csv')
put_file(credentials_10,'tfid_test.csv')

<Response [201]>
<Response [201]>
<Response [201]>


IOError: [Errno 2] No such file or directory: 'tfid_test.csv'

Data Cleaning part1 done

<h3>Extracting Details from Description</h3>

In [ ]:
target=train.Is_Response
train.drop(['Is_Response'],inplace=True,axis=1)
df = train.append(test)

In [ ]:
df=df.assign(description_len = train.Description.str.len())

In [ ]:
tok_and_tag = lambda x: nltk.pos_tag(nltk.word_tokenize(x.decode('utf-8')))
df.loc[:,'tagged_desc']=df['Description'].apply(tok_and_tag)

In [ ]:
#getting possible tags
tokens, tags = zip(*chain(*df['tagged_desc'].tolist()))
possible_tags = sorted(set(tags))
df.loc[:,'pos_counts'] = df['tagged_desc'].apply(lambda x: Counter(list(zip(*x))[1]))

In [ ]:
##Adding zeros to tages not present 
def add_pos_with_zero_counts(counter, keys_to_add):
    for k in keys_to_add:
        counter[k] = counter.get(k, 0)
    return counter
df.loc[:,'pos_counts_with_zero'] =df['pos_counts'].apply(lambda x: add_pos_with_zero_counts(x, possible_tags))

In [ ]:
df['sent_vector'] =df['pos_counts_with_zero'].apply(lambda x: [count for tag, count in sorted(x.most_common())])

In [ ]:
dftrain= train.append(test,ignore_index=True)

In [ ]:
df2 = pd.DataFrame(df['sent_vector'].tolist())
df2.columns = possible_tags
df2['project_id']=dftrain.project_id

In [ ]:
nltk_final =pd.merge(df, df2, left_on='project_id', right_on='project_id')

In [ ]:
put_file(,'FTRL6.csv')